# Modeling and Simulation in Python

Chapter 22

Copyright 2017 Allen Downey

License: [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0)


In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

### Vectors

A `Vector` object represents a vector quantity.  In the context of mechanics, vector quantities include position, velocity, acceleration, and force, all of which might be in 2D or 3D.

You can define a `Vector` object without units, but if it represents a physical quantity, you will often want to attach units to it.

I'll start by grabbing the units we'll need.

In [2]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram

Here's a two dimensional `Vector` in meters.

In [3]:
A = Vector(3, 4) * m

We can access the elements by name.

In [4]:
A.x

In [5]:
A.y

The magnitude is the length of the vector.

In [6]:
A.mag

The angle is the number of radians between the vector and the positive x axis.

In [7]:
A.angle

If we make another `Vector` with the same units,

In [8]:
B = Vector(1, 2) * m

We can add `Vector` objects like this

In [9]:
A + B

And subtract like this:

In [10]:
A - B

We can compute the Euclidean distance between two Vectors.

In [11]:
A.dist(B)

And the difference in angle

In [13]:
A.diff_angle(B)

If we are given the magnitude and angle of a vector, what we have is the representation of the vector in polar coordinates.

In [14]:
mag = A.mag
angle = A.angle

We can use `pol2cart` to convert from polar to Cartesian coordinates, and then use the Cartesian coordinates to make a `Vector` object.

In this example, the `Vector` we get should have the same components as `A`.

In [15]:
x, y = pol2cart(angle, mag)
Vector(x, y)

Another way to represent the direction of `A` is a unit vector, which is a vector with magnitude 1 that points in the same direction as `A`.  You can compute a unit vector by dividing a vector by its magnitude:

In [16]:
A / A.mag

Or by using the `hat` function, so named because unit vectors are conventionally decorated with a hat, like this: $\hat{A}$:

In [17]:
A.hat()

**Exercise:** Create a `Vector` named `a_grav` that represents acceleration due to gravity, with x component 0 and y component $-9.8$ meters / second$^2$.

In [18]:
# Solution goes here

### Degrees and radians

Pint provides units to represent degree and radians.

In [20]:
degree = UNITS.degree
radian = UNITS.radian

If you have an angle in degrees,

In [21]:
angle = 45 * degree
angle

You can convert to radians.

In [22]:
angle_rad = angle.to(radian)

If it's already in radians, `to` does the right thing.

In [23]:
angle_rad.to(radian)

You can also convert from radians to degrees.

In [24]:
angle_rad.to(degree)

As an alterative, you can use `np.deg2rad`, which works with Pint quantities, but it also works with simple numbers and NumPy arrays:

In [25]:
np.deg2rad(angle)

**Exercise:** Create a `Vector` named `a_force` that represents acceleration due to a force of 0.5 Newton applied to an object with mass 0.3 kilograms, in a direction 45 degrees up from the positive x-axis.

Add `a_force` to `a_grav` from the previous exercise.  If that addition succeeds, that means that the units are compatible.  Confirm that the total acceleration seems to make sense.

In [26]:
# Solution goes here

In [27]:
# Solution goes here

### Baseball

Here's a `Params` object that contains parameters for the flight of a baseball.

In [28]:
params = Params(x = 0 * m, 
                y = 1 * m,
                g = 9.8 * m/s**2,
                mass = 145e-3 * kg,
                diameter = 73e-3 * m,
                rho = 1.2 * kg/m**3,
                C_d = 0.33,
                angle = 45 * degree,
                velocity = 40 * m / s,
                t_end = 10 * s)

And here's the function that uses the `Params` object to make a `System` object.

In [29]:
def make_system(params):
    """Make a system object.
    
    params: Params object with angle, velocity, x, y,
               diameter, duration, g, mass, rho, and C_d
               
    returns: System object
    """
    unpack(params)
    
    # convert angle to degrees
    theta = np.deg2rad(angle)
    
    # compute x and y components of velocity
    vx, vy = pol2cart(theta, velocity)
    
    # make the initial state
    init = State(x=x, y=y, vx=vx, vy=vy)
    
    # compute area from diameter
    area = np.pi * (diameter/2)**2
    
    return System(params, init=init, area=area)

Here's how we use it:

In [31]:
system = make_system(params)

Here's a function that computes drag force using vectors:

In [32]:
def drag_force(v, system):
    """Computes drag force in the opposite direction of `v`.
    
    v: velocity Vector
    system: System object with rho, C_d, area
    
    returns: Vector drag force
    """
    unpack(system)
    mag = -rho * v.mag**2 * C_d * area / 2
    direction = v.hat()
    f_drag = direction * mag
    return f_drag

We can test it like this.

In [35]:
v_test = Vector(10, 10) * m/s
drag_force(v_test, system)

Here's the slope function that computes acceleration due to gravity and drag.

In [36]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)

    v = Vector(vx, vy)    
    a_drag = drag_force(v, system) / mass
    a_grav = Vector(0, -g)
    
    a = a_grav + a_drag
    
    return vx, vy, a.x, a.y

Always test the slope function with the initial conditions.

In [37]:
slope_func(system.init, 0, system)

We can use an event function to stop the simulation when the ball hits the ground:

In [38]:
def event_func(state, t, system):
    """Stop when the y coordinate is 0.
    
    state: State object
    t: time
    system: System object
    
    returns: y coordinate
    """
    x, y, vx, vy = state
    return y

In [39]:
event_func(system.init, 0, system)

Now we can call `run_ode_solver`

In [40]:
results, details = run_ode_solver(system, slope_func, events=event_func, max_step=0.2*s)
details

The final label tells us the flight time.

In [41]:
flight_time = get_last_label(results) * s

The final value of `x` tells us the how far the ball landed from home plate:

In [42]:
x_dist = get_last_value(results.x) * m

### Visualizing the results

The simplest way to visualize the results is to plot x and y as functions of time.

In [43]:
plot(results.x, label='x')
plot(results.y, label='y')

decorate(xlabel='Time (s)',
         ylabel='Position (m)')

savefig('figs/chap10-fig01.pdf')

We can plot the velocities the same way.

In [44]:
plot(results.vx, label='vx')
plot(results.vy, label='vy')

decorate(xlabel='Time (s)',
         ylabel='Velocity (m/s)')

The x velocity slows down due to drag.

The y velocity drops quickly while drag and gravity are in the same direction, then more slowly after the ball starts to fall.

Another way to visualize the results is to plot y versus x.  The result is the trajectory of the ball through its plane of motion.

In [45]:
def plot_trajectory(results):
    plot(results.x, results.y, label='trajectory')

    decorate(xlabel='x position (m)',
             ylabel='y position (m)')

plot_trajectory(results)
savefig('figs/chap10-fig02.pdf')

### Under the hood

`Vector` is a function that returns a `ModSimVector` object.

In [46]:
v = Vector(3, 4)
type(v)

A `ModSimVector` is a specialized kind of Pint `Quantity`.

In [47]:
isinstance(v, Quantity)

There's one gotcha you might run into with Vectors and Quantities.  If you multiply a `Vector` and a `Quantity`, you get a `Vector`:

In [48]:
v1 = v * m

In [49]:
type(v1)

But if you multiply a `Quantity` and a `Vector`, you get a `Quantity`:

In [50]:
v2 = m * v

In [51]:
type(v2)

With a `Vector` you can get the coordinates using dot notation, as well as `mag`, `mag2`, and `angle`:

In [52]:
v1.x, v1.y, v1.mag, v1.angle

With a `Quantity`, you can't.  But you can use indexing to get the coordinates:

In [53]:
v2[0], v2[1]

And you can use vector functions to get the magnitude and angle.

In [54]:
vector_mag(v2), vector_angle(v2)

And often you can avoid the whole issue by doing the multiplication with the `Vector` on the left.

### Exercises

**Exercise:** Run the simulation for a few different launch angles and visualize the results.  Are they consistent with your expectations?

**Exercise:** The baseball stadium in Denver, Colorado is 1,580 meters above sea level, where the density of air is about 1.0 kg / meter$^3$.  How much farther would a ball hit with the same velocity and launch angle travel?

Hint: create a new `Params` object like this:

In [55]:
params2 = Params(params, rho=1*kg/m**3)

In [56]:
# Solution goes here

In [57]:
# Solution goes here

**Exercise:** The model so far is based on the assumption that coefficient of drag does not depend on velocity, but in reality it does.  The following figure, from Adair, [*The Physics of Baseball*](https://books.google.com/books/about/The_Physics_of_Baseball.html?id=4xE4Ngpk_2EC), shows coefficient of drag as a function of velocity.

<img src="data/baseball_drag.png" width="400">


I used [an online graph digitizer](https://automeris.io/WebPlotDigitizer/) to extract the data and save it in a CSV file.  Here's how we can read it:

In [58]:
# Solution goes here

Modify the model to include the dependence of `C_d` on velocity, and see how much it affects the results.  Hint: use `interpolate`.

In [59]:
# Solution goes here

In [62]:
# Solution goes here

In [63]:
# Solution goes here

In [64]:
# Solution goes here

In [65]:
# Solution goes here

In [66]:
# Solution goes here

In [67]:
# Solution goes here